In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import metrics

from pprint import pprint
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

C:\Users\Minus\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load the entire dataset
data = pd.read_csv('../Datasets/results.csv', header = 0, names = ['Query', 'Job Title', 'Description'])

#Split data to train and test (80 - 20)

train, test = train_test_split(data, test_size = 0.2)

train_descs = train['Description']
train_labels = train['Query']
#train_labels = train['Job Title']
 
test_descs = test['Description']
test_labels = test['Query']
#test_labels = test['Job Title']

print(len(data['Query'].unique().tolist()))
print(len(data['Job Title'].unique().tolist()))

print(len(train_labels.unique().tolist()))
print(len(test_labels.unique().tolist()))

82
1214
81
81


In [3]:
num_labels = len(train_labels.unique().tolist())
vocab_size = 1000
batch_size = 100
nb_epoch = 30

# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_descs)
 
x_train = tokenizer.texts_to_matrix(train_descs, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_descs, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [7]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam', # or 'sgd'
              metrics = [metrics.categorical_accuracy, 'accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 81)                41553     
__________

In [8]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_split=0.1)

Train on 1080 samples, validate on 120 samples
Epoch 1/30
1080/1080 [==============================] - 1s 481us/step - loss: 4.5932 - categorical_accuracy: 0.0269 - acc: 0.0269 - val_loss: 4.1159 - val_categorical_accuracy: 0.1083 - val_acc: 0.1083
Epoch 2/30
1080/1080 [==============================] - 0s 230us/step - loss: 3.5811 - categorical_accuracy: 0.2167 - acc: 0.2167 - val_loss: 3.6816 - val_categorical_accuracy: 0.2333 - val_acc: 0.2333
Epoch 3/30
1080/1080 [==============================] - 0s 232us/step - loss: 2.7068 - categorical_accuracy: 0.4278 - acc: 0.4278 - val_loss: 3.1254 - val_categorical_accuracy: 0.3167 - val_acc: 0.3167
Epoch 4/30
1080/1080 [==============================] - 0s 230us/step - loss: 1.9778 - categorical_accuracy: 0.5648 - acc: 0.5648 - val_loss: 2.9324 - val_categorical_accuracy: 0.3250 - val_acc: 0.3250
Epoch 5/30
1080/1080 [==============================] - 0s 233us/step - loss: 1.3175 - categorical_accuracy: 0.7130 - acc: 0.7130 - val_loss: 2.8

In [10]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('\nTest categorical_crossentropy:', score[0])
print('Categorical accuracy:', score[1])
print('Accuracy:', score[2])


300/300 [==============================] - 0s 80us/step

Test categorical_crossentropy: 3.394968827565511
Categorical accuracy: 0.33666666348775226
Accuracy: 0.33666666348775226
